# gensim tfidf

Tips of using gensim Tfidf_Vec<br/>
_________
### steps:<br />
* 1 docs ---> dictionary  <br/>
. a. docs is a list consist of docutmnet <br/>
. b. use the ```jieba.cut(doc, cut_all=False)``` segmenting list<br/>
. c. use the gensim ```corpora.Dictionary(words)``` to construst a dictionary<br/>
. d. dictionary has many attributes(cfs,dfs,token2id...)<br/>
. e. use the new variable ```new_corpus= [dictionary.doc2bow(doc) for doc in docs]``` <br/>
* 2 dictionary --> gensim Tfidf model<br/>
.a. from gensim import models<br/>
.b. acquire tfidf training model```models.TfidfModel(new_corpus)```<br/>
.c. fun using the model to acquire vector of tfidf ``` tfidf_model[string_bow] ```<br/>

In [1]:
from gensim.sklearn_api import TfIdfTransformer
from gensim import corpora

In [2]:
from gensim.corpora import Dictionary

In [3]:
words = [['红色', '的', '水果', '是', '什么'],
 [ '黎明', '中间', '是', '我'],
 ['我', '在', '小红', '和','黎明','之间'],
 ['是', '那里', '在', '我', '你']]

In [4]:
dictionary = corpora.Dictionary(words)

In [5]:
new_corpus = [dictionary.doc2bow(t) for t in words]

In [6]:
from gensim import models
tfidf = models.TfidfModel(new_corpus)

# $ idf = add + log_{log\_base} \frac{total~docs}{doc~freq}$

# Python实现TF-IDF

In [17]:
from collections import Counter

In [18]:
import numpy as np

In [19]:
x = np.array([['a','b','b','c'],['c','d','e','b','f'],['s','a','d','e'],['a','e','s','w','c']])

In [20]:
Counter(x[0])

Counter({'a': 1, 'b': 2, 'c': 1})

In [21]:
ss = {'a':1,'b':2,'c':3}

In [22]:
s = np.sum(list(Counter(ss).values()))

In [23]:
def custom_wlocal(x):

    ll = dict() # term frequency
    for i, m in Counter(x).items():
        ll[i] = Counter(x)[i] / np.sum(list(Counter(x).values()))
    return ll

In [24]:
Counter(x[0])['b']

2

In [25]:
custom_wlocal(ss)

{'a': 0.16666666666666666, 'b': 0.3333333333333333, 'c': 0.5}

In [26]:
from pandas import DataFrame

In [27]:
from collections import defaultdict
from math import log10

In [28]:
def custom_wglobal(x):
    # 遍历一遍，拿到所有的单词
    words = set()
    for n in x:
        for i in n:
            words.add(i)
    #针对每个不重复单词尽心文档的计数
    doc = defaultdict(float)
    try:
        for word in words:
            # 查找每个文档
            for n in x:
                if word in n:
                    doc[word] += 1
            #print(doc[word])
            doc[word] = log10(len(x) / (1.0 + doc[word]))
    except Exception as e:
        print(e)
    return doc

In [29]:
custom_wglobal(x)

defaultdict(float,
            {'f': 0.3010299956639812,
             'e': 0.0,
             'c': 0.0,
             'w': 0.3010299956639812,
             'd': 0.12493873660829993,
             'a': 0.0,
             'b': 0.12493873660829993,
             's': 0.12493873660829993})

## 获得TfIdf值

In [30]:
# 每篇文档的TF-IDF值d
def avg_tfidf(x, tf, idf):
    # 每篇文档tf
    Tf = [tf(tf_id) for tf_id in x]
    # 总idf
    Idf = idf(x)
    # 计算每篇的TF-IDF值
    text_of_tfidf = defaultdict(float)
    doc_of_tfidf = defaultdict(list)
    h = 0
    for doc in Tf:
        for i, m in doc.items():
            if i in Idf:
                text_of_tfidf[i] = doc[i] * Idf[i]
                
        h += 1
        doc_of_tfidf['doc'+ str(h)] = text_of_tfidf
        text_of_tfidf = defaultdict(float)
    return DataFrame(doc_of_tfidf).fillna(0)

In [31]:
dataframe = avg_tfidf(x, custom_wlocal, custom_wglobal)

In [32]:
dataframe

,doc1,doc2,doc3,doc4
a,0.000000,0.000000,0.000000,0.000000
b,0.062469,0.024988,0.000000,0.000000
c,0.000000,0.000000,0.000000,0.000000
d,0.000000,0.024988,0.031235,0.000000
e,0.000000,0.000000,0.000000,0.000000
f,0.000000,0.060206,0.000000,0.000000
s,0.000000,0.000000,0.031235,0.024988
w,0.000000,0.000000,0.000000,0.060206


In [33]:
text1 = '2019年国家网络安全宣传周开幕式16日在天津举行。中共中央政治局委员、中宣部部长黄坤明在开幕式上宣读习近平的重要指示并讲话。他说，要认真学习贯彻习近平总书记重要指示精神，深刻把握信息化发展大势，积极应对网络安全挑战，充分发挥广大人民在维护网络安全中的主体作用，把“四个坚持”的原则要求落到实处，有力维护人民群众在网络空间的切身利益。'

In [34]:
import jieba, re

In [35]:
def cut(string):
    regex = r"(\d+|\w+)"
    ret = re.findall(regex,string=string)
    words = list()
    for p in ret:
        words += list(jieba.cut(p, cut_all=False))
    #[list(jieba.cut(p, cut_all=False)) for p in ret]
    return words
#list(jieba.cut(string, cut_all=False))

In [36]:
np.sum(list(custom_wlocal(cut(text1)).values()))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\C00495~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.662 seconds.
Prefix dict has been built succesfully.


1.0000000000000002

In [37]:
text2 = "据介绍，第二次演练包括庆祝大会仪式、阅兵、群众游行、联欢活动、转场及应急处置6项演练内容，重点磨合内部衔接，全面检验了各方组织指挥和保障体系运行工作。"

In [38]:
list(custom_wlocal(cut(text2)).items())

[('据介绍', 0.030303030303030304),
 ('第二次', 0.030303030303030304),
 ('演练', 0.06060606060606061),
 ('包括', 0.030303030303030304),
 ('庆祝大会', 0.030303030303030304),
 ('仪式', 0.030303030303030304),
 ('阅兵', 0.030303030303030304),
 ('群众', 0.030303030303030304),
 ('游行', 0.030303030303030304),
 ('联欢', 0.030303030303030304),
 ('活动', 0.030303030303030304),
 ('转场', 0.030303030303030304),
 ('及', 0.030303030303030304),
 ('应急', 0.030303030303030304),
 ('处置', 0.030303030303030304),
 ('6', 0.030303030303030304),
 ('项', 0.030303030303030304),
 ('内容', 0.030303030303030304),
 ('重点', 0.030303030303030304),
 ('磨合', 0.030303030303030304),
 ('内部', 0.030303030303030304),
 ('衔接', 0.030303030303030304),
 ('全面', 0.030303030303030304),
 ('检验', 0.030303030303030304),
 ('了', 0.030303030303030304),
 ('各方', 0.030303030303030304),
 ('组织', 0.030303030303030304),
 ('指挥', 0.030303030303030304),
 ('和', 0.030303030303030304),
 ('保障体系', 0.030303030303030304),
 ('运行', 0.030303030303030304),
 ('工作', 0.030303030303030304)]

In [39]:
text3 = "近日，新疆军区某陆航旅组织部队采取空地一体的方式，机动至天山北麓某地域，展开了多机型、多弹种、多靶型跨昼夜火力突击演练。"

In [40]:
text4 = "夜幕降临，飞行员驾驶直升机再次升空，接受攻击指令，采取红外瞄准的方式，发射导弹，对目标进行精准火力打击摧毁，并释放干扰弹。在持续进行了14个小时的大强度饱和攻击状态后，实弹射击演练结束"

In [41]:
text = [text1,text2, text3, text4]
tmp_text = [cut(t) for t in text]

In [42]:
len(tmp_text)

4

In [43]:
word_list = tmp_text

In [44]:
from gensim import corpora
# 赋给语料库中每个词(不重复的词)一个整数id
dictionary = corpora.Dictionary(word_list)
new_corpus = [dictionary.doc2bow(text) for text in word_list]
print(new_corpus)

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 2), (9, 2), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 4), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 2), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 4), (39, 1), (40, 2), (41, 3), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 2), (53, 1)], [(43, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 2), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1)], [(38, 1), (55, 1), (74, 1), (77, 1), (85, 1), (86, 1), (87, 1), (88, 2), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 1), (97, 1), (98, 2), (99, 1), (100, 1), (101, 1), (102, 1), (103, 1), (104, 1), (105, 1),

In [45]:
corpus = [
    'this is the first document',
    'this is the second second document',
    'and the third one',
    'is this the first document'
]

In [46]:
word_list = []
for i in range(len(corpus)):
    word_list.append(corpus[i].split(' '))

In [47]:
from gensim import models
tfidf = models.TfidfModel(new_corpus)
tfidf.save("my_model.tfidf")

# 载入模型
tfidf = models.TfidfModel.load("my_model.tfidf")

# 使用这个训练好的模型得到单词的tfidf值
tfidf_vec = []
for i in range(len(corpus)):
    string = corpus[i]
    string_bow = dictionary.doc2bow(string.lower().split())
    string_tfidf = tfidf[string_bow]
    tfidf_vec.append(string_tfidf)
print(tfidf_vec)

[[], [], [], []]


In [48]:
data = avg_tfidf(tmp_text, custom_wlocal, custom_wglobal)

In [49]:
from math import sqrt

In [50]:
def vec_of_tfidf(dataframe):
    num = dataframe.shape[1]
    for i in range(num): 
        for j in range(1,num):
            fenzi = np.dot(dataframe.iloc[i], dataframe.iloc[j])
            fenmu = (sqrt(np.dot(dataframe.iloc[i],dataframe.iloc[i]))*sqrt(np.dot(dataframe.iloc[j],dataframe.iloc[j])))
        
        #cos_vec_tdidf = fenzi / fenmu
            print(i, i+1,fenzi/fenmu)

In [51]:
vec_of_tfidf(data)  # 4个文本的cos值

0 1 0.0
0 1 0.0
0 1 0.0
1 2 1.0
1 2 1.0
1 2 0.0
2 3 1.0
2 3 1.0
2 3 0.0
3 4 0.0
3 4 0.0
3 4 1.0
